# Factorization Machine (FM) Recommender System
This notebook implements a 2-way Factorization Machine from scratch using NumPy.
It evaluates the model using both **RMSE** (rating accuracy) and **NDCG** (ranking quality).

### Libraries Import

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, ndcg_score
from tqdm import tqdm
import kagglehub

print("Libraries imported successfully.")

Libraries imported successfully.


/home/rami/git/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [2]:
path = kagglehub.dataset_download("odedgolden/movielens-1m-dataset")

ratings_cols = ['UserID', 'MovieID', 'Rating', 'Timestamp']
movies_cols = ['MovieID', 'Title']

ratings_df = pd.read_csv(os.path.join(path, 'ratings.dat'), sep='::', names=ratings_cols, encoding='latin-1', engine='python')
movies_df = pd.read_csv(os.path.join(path, 'movies.dat'), sep='::', names=movies_cols, usecols=[0, 1], encoding='latin-1', engine='python')

print(f"Loaded {len(ratings_df)} ratings and {len(movies_df)} movies.")

Loaded 1000209 ratings and 3883 movies.


### Data Preparation & Splitting

In [3]:

# Global Mappings (Shared between Train & Test)
# This ensures consistent Feature IDs across both sets.
user_map = {id_: idx for idx, id_ in enumerate(sorted(ratings_df['UserID'].unique()))}
movie_map = {id_: idx for idx, id_ in enumerate(sorted(ratings_df['MovieID'].unique()))}

n_users = len(user_map)
n_movies = len(movie_map)
n_features = n_users + n_movies

print(f"Total Users: {n_users}, Total Movies: {n_movies}, Total Features: {n_features}")

# Split DATAFRAME first (Required for NDCG user grouping)
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)
print(f"Train samples: {len(train_df)}, Test samples: {len(test_df)}")

# Conversion Function (DataFrame -> Sparse Matrix)
def df_to_sparse_matrix(df, user_map, movie_map, n_users, n_features):
    """Converts a ratings DataFrame to a sparse feature matrix X and target y."""
    n_samples = df.shape[0]
    X = lil_matrix((n_samples, n_features), dtype=np.int8)
    y = df['Rating'].values.astype(np.float32)

    for i, row in enumerate(tqdm(df.itertuples(index=False), total=n_samples, desc="Building Matrix")):
        # User one-hot feature
        X[i, user_map[row.UserID]] = 1
        # Movie one-hot feature (offset by number of users)
        X[i, n_users + movie_map[row.MovieID]] = 1

    return X.tocsr(), y # Convert to CSR for fast arithmetic later


print("\nBuilding Training Matrix...")
X_train, y_train = df_to_sparse_matrix(train_df, user_map, movie_map, n_users, n_features)
print("Building Testing Matrix...")
X_test, y_test = df_to_sparse_matrix(test_df, user_map, movie_map, n_users, n_features)

Total Users: 6040, Total Movies: 3706, Total Features: 9746
Train samples: 800167, Test samples: 200042

Building Training Matrix...


Building Matrix: 100%|██████████| 800167/800167 [00:02<00:00, 362719.37it/s]


Building Testing Matrix...


Building Matrix: 100%|██████████| 200042/200042 [00:00<00:00, 370419.72it/s]


### Factorization Machine Model Implementation

In [4]:

class FactorizationMachine:
    """
    A 2-Way Factorization Machine implemented with NumPy.
    Optimized for sparse inputs and vectorized batch predictions.
    """
    def __init__(self, n_features, k=10, learning_rate=0.01, l2_reg=0.1, n_epochs=5):
        self.n_features = n_features
        self.k = k
        self.lr = learning_rate
        self.l2 = l2_reg
        self.n_epochs = n_epochs

        # Parameters
        self.w0 = 0.0
        self.w = np.zeros(n_features)
        self.V = np.random.normal(0, 0.1, (n_features, k))

        # For early stopping / best model saving
        self.best_test_rmse = float('inf')
        self.best_w0 = self.w0
        self.best_w = self.w.copy()
        self.best_V = self.V.copy()

    def predict_batch(self, X):
        """Optimized batch prediction for CSR matrix X."""
        n_samples = X.shape[0]
        # Linear part (Global bias + User bias + Movie bias)
        pred = np.full(n_samples, self.w0) + X.dot(self.w)
        
        # Interaction part (User vector dot Movie vector)
        if self.k > 0:
            # Efficient calculation using FM identity: 0.5 * [ (sum(Vi*xi))^2 - sum((Vi*xi)^2) ]
            term1 = (X.dot(self.V)) ** 2
            term2 = X.dot(self.V ** 2)
            interaction = 0.5 * np.sum(term1 - term2, axis=1)
            pred += interaction
        return pred

    def fit(self, X_train, y_train, X_test, y_test):
        """Trains the model using Stochastic Gradient Descent (SGD)."""
        print("\nStarting training...")
        train_indices = np.arange(X_train.shape[0])

        for epoch in range(self.n_epochs):
            np.random.shuffle(train_indices)
            
            for idx in tqdm(train_indices, desc=f"Epoch {epoch+1}/{self.n_epochs}", leave=False):
                x = X_train[idx]
                y_true = y_train[idx]

                # --- SGD Step (Optimized for single sample) ---
                active_idx = x.nonzero()[1]
                
                # Forward pass
                pred = self.w0 + np.sum(self.w[active_idx])
                if self.k > 0:
                    v_active = self.V[active_idx]
                    sum_v = np.sum(v_active, axis=0)
                    interaction = 0.5 * np.sum(sum_v**2 - np.sum(v_active**2, axis=0))
                    pred += interaction

                error = pred - y_true

                # Backward pass (Updates)
                self.w0 -= self.lr * error
                
                w_grad = error + self.l2 * self.w[active_idx]
                self.w[active_idx] -= self.lr * w_grad

                if self.k > 0:
                    v_sum = np.sum(self.V[active_idx], axis=0)
                    # Gradient for V uses the pre-calculated sum for efficiency
                    v_grad = error * (v_sum - self.V[active_idx]) + self.l2 * self.V[active_idx]
                    self.V[active_idx] -= self.lr * v_grad

            # Evaluation at end of epoch
            train_rmse = self.evaluate_rmse(X_train, y_train)
            test_rmse = self.evaluate_rmse(X_test, y_test)
            print(f"Epoch {epoch+1}: Train RMSE: {train_rmse:.4f} | Test RMSE: {test_rmse:.4f}")

            # Save best model
            if test_rmse < self.best_test_rmse:
                self.best_test_rmse = test_rmse
                self.best_w0 = self.w0
                self.best_w = self.w.copy()
                self.best_V = self.V.copy()
                print(f" -> New best model saved (Test RMSE: {test_rmse:.4f})")

        # Restore best model
        self.w0 = self.best_w0
        self.w = self.best_w
        self.V = self.best_V
        print("Training complete. Best model restored.")

    def evaluate_rmse(self, X, y_true):
        y_pred = self.predict_batch(X)
        # Clip predictions to valid rating range [1, 5] for realistic RMSE
        y_pred = np.clip(y_pred, 1.0, 5.0)
        return np.sqrt(mean_squared_error(y_true, y_pred))

### NDCG Evaluation Function

In [5]:

def evaluate_ndcg(model, test_df, user_map, movie_map, n_users, k=10):
    """
    Calculates average NDCG@k for users in the test set.
    This measures RANKING quality (how well top recommendations match true preferences).
    """
    print(f"\nCalculating NDCG@{k} on Test Set...")
    ndcg_scores = []
    
    # Group test data by user to evaluate their personal ranking
    grouped = test_df.groupby('UserID')
    
    for user_id, group in tqdm(grouped, desc="Evaluated Users"):
        # Need at least 2 items to have a meaningful ranking
        if len(group) < 2: continue

        y_true = group['Rating'].values
        y_true_2d = [y_true]

        # Prepare features for this user's test items
        u_idx = user_map[user_id]
        m_indices = [movie_map[mid] for mid in group['MovieID'].values]
        n_samples = len(group)

        # sparse matrix for this small batch
        rows = np.concatenate([np.arange(n_samples), np.arange(n_samples)])
        cols = np.concatenate([np.full(n_samples, u_idx), np.array(m_indices) + n_users])
        data = np.ones(len(rows))
        X_user = csr_matrix((data, (rows, cols)), shape=(n_samples, model.n_features))

        # Predict ranking scores
        y_pred = model.predict_batch(X_user)
        y_pred_2d = [y_pred]

        # Calculate NDCG@k for this user
        score = ndcg_score(y_true_2d, y_pred_2d, k=k)
        ndcg_scores.append(score)

    avg_ndcg = np.mean(ndcg_scores)
    print(f"Average NDCG@{k}: {avg_ndcg:.4f}")
    return avg_ndcg

### Train & Evaluate

In [6]:
fm = FactorizationMachine(
    n_features=n_features,
    k=15,                 # Latent dimension size
    learning_rate=0.005, 
    l2_reg=0.1, 
    n_epochs=5
)

fm.fit(X_train, y_train, X_test, y_test)

print(f"\nFinal Test RMSE: {fm.best_test_rmse:.4f}")

final_ndcg = evaluate_ndcg(fm, test_df, user_map, movie_map, n_users, k=10)


Starting training...


Epoch 1: Train RMSE: 0.9340 | Test RMSE: 0.9441
 -> New best model saved (Test RMSE: 0.9441)


Epoch 2: Train RMSE: 0.9174 | Test RMSE: 0.9294
 -> New best model saved (Test RMSE: 0.9294)


Epoch 3: Train RMSE: 0.9087 | Test RMSE: 0.9220
 -> New best model saved (Test RMSE: 0.9220)


Epoch 4: Train RMSE: 0.9064 | Test RMSE: 0.9201
 -> New best model saved (Test RMSE: 0.9201)


Epoch 5: Train RMSE: 0.9031 | Test RMSE: 0.9173
 -> New best model saved (Test RMSE: 0.9173)
Training complete. Best model restored.

Final Test RMSE: 0.9173

Calculating NDCG@10 on Test Set...


Evaluated Users: 100%|██████████| 6038/6038 [00:03<00:00, 1728.15it/s]

Average NDCG@10: 0.9107


### Test Recommendations

In [7]:
def get_top_n_recommendations(user_id_original, n_recommends, fm_model, movies_df, ratings_df, user_map, movie_map, n_users):
    """Generates top N movie recommendations for an existing user using batch prediction."""
    if user_id_original not in user_map:
        print(f"User ID {user_id_original} not found.")
        return None
    u_idx = user_map[user_id_original]

    # unseen movies
    seen_ids = ratings_df[ratings_df['UserID'] == user_id_original]['MovieID'].unique()
    all_ids = movies_df['MovieID'].unique()
    unseen_ids = np.setdiff1d(all_ids, seen_ids)
    unseen_ids = [mid for mid in unseen_ids if mid in movie_map]
    n_unseen = len(unseen_ids)
    
    if n_unseen == 0: return None

    # Unique row index for each prediction
    row_indices = np.arange(n_unseen) 
    user_col_indices = np.full(n_unseen, u_idx)
    movie_col_indices = np.array([movie_map[mid] for mid in unseen_ids]) + n_users

    # sparse matrix
    rows = np.concatenate([row_indices, row_indices])
    cols = np.concatenate([user_col_indices, movie_col_indices])
    data_vals = np.ones(len(rows), dtype=np.int8)
    X_pred = csr_matrix((data_vals, (rows, cols)), shape=(n_unseen, fm_model.n_features))

    # Predict & Sort
    predicted_ratings = fm_model.predict_batch(X_pred)
    movie_rating_pairs = sorted(zip(unseen_ids, predicted_ratings), key=lambda x: x[1], reverse=True)

    # Get top N details
    top_n_ids = [mid for mid, _ in movie_rating_pairs[:n_recommends]]
    return movies_df.set_index('MovieID').loc[top_n_ids].reset_index()

def get_cold_start_recommendations(n_recommends, ratings_df, movies_df):
    """Popularity-based recommendations for new users."""
    print("Generating cold-start recommendations...")
    top_popular_ids = ratings_df['MovieID'].value_counts().head(n_recommends).index
    return movies_df.set_index('MovieID').loc[top_popular_ids].reset_index()

In [8]:

user_id_existing = 4
print(f"--- Recommendations for User {user_id_existing} ---")
print(get_top_n_recommendations(user_id_existing, 5, fm, movies_df, ratings_df, user_map, movie_map, n_users))

print(f"\n--- Cold Start Recommendations ---")
print(get_cold_start_recommendations(5, ratings_df, movies_df))

--- Recommendations for User 4 ---
   MovieID                                              Title
0     2019  Seven Samurai (The Magnificent Seven) (Shichin...
1      318                   Shawshank Redemption, The (1994)
2      745                              Close Shave, A (1995)
3     1148                         Wrong Trousers, The (1993)
4      922      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)

--- Cold Start Recommendations ---
Generating cold-start recommendations...
   MovieID                                              Title
0     2858                             American Beauty (1999)
1      260          Star Wars: Episode IV - A New Hope (1977)
2     1196  Star Wars: Episode V - The Empire Strikes Back...
3     1210  Star Wars: Episode VI - Return of the Jedi (1983)
4      480                               Jurassic Park (1993)
